##Install and Import the Required Packages

In [ ]:
!pip install diffusers transformers accelerate controlnet_aux datasets xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.6/274.6 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.0/510.0 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 2.0 MB/s eta 0:00:00
  Using cached

In [ ]:
!git clone https://github.com/ArefMYTB/IP-Adapter

Cloning into 'IP-Adapter'...
remote: Enumerating objects: 292, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 292 (delta 105), reused 90 (delta 90), pack-reused 171
Receiving objects: 100% (292/292), 69.56 MiB | 31.53 MiB/s, done.
Resolving deltas: 100% (150/150), done.


In [ ]:
%cd IP-Adapter

/content/IP-Adapter


In [ ]:
from diffusers import StableDiffusionControlNetInpaintPipeline, ControlNetModel, UniPCMultistepScheduler
from diffusers.pipelines.controlnet.pipeline_controlnet_inpaint import *
from diffusers.utils import load_image

from ip_adapter import IPAdapter

from PIL import Image
import torch

import os

In [ ]:
%cd ip_adapter

/content/IP-Adapter/ip_adapter


In [ ]:
!git lfs install
!git clone https://huggingface.co/h94/IP-Adapter

Updated git hooks.
Git LFS initialized.
Cloning into 'IP-Adapter'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 101 (delta 26), reused 6 (delta 6), pack-reused 37 (from 1)
Receiving objects: 100% (101/101), 398.43 KiB | 14.76 MiB/s, done.
Resolving deltas: 100% (26/26), done.
Filtering content: 100% (25/25), 18.45 GiB | 25.24 MiB/s, done.


In [ ]:
!cp -r IP-Adapter/models ../
!rm -rf IP-Adapter

In [ ]:
%cd ..

/content/IP-Adapter


##Download IP-Adapter Pretrained Weights

In [ ]:
!gdown https://civitai.com/api/download/models/245627?type=Model&format=SafeTensor&size=pruned&fp=fp16

Downloading...
From: https://civitai.com/api/download/models/245627?type=Model
To: /content/IP-Adapter/245627?type=Model
100% 2.13G/2.13G [00:32<00:00, 66.4MB/s]


##Unzip the Uploaded Dataset

In [ ]:
!unzip /content/NTED.zip -d /content

Archive:  /content/NTED.zip
   creating: /content/NTED/
   creating: /content/NTED/1/
  inflating: /content/NTED/1/distorted.jpg  
  inflating: /content/NTED/1/flawless.jpg  
  inflating: /content/NTED/1/mask.jpg  
  inflating: /content/NTED/1/segment.png  


##Initiate the Model

In [ ]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

device

device(type='cuda', index=0)

In [ ]:
canny_controlnet = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16)
pose_controlnet = ControlNetModel.from_pretrained("NegarMov/controlnet-pose-finetuned", torch_dtype=torch.float16)
segment_controlnet = ControlNetModel.from_pretrained("NegarMov/controlnet-seg-finetuned", torch_dtype=torch.float16)

controlnet = [canny_controlnet, pose_controlnet, segment_controlnet]

pipe = StableDiffusionControlNetInpaintPipeline.from_single_file(
    "/content/IP-Adapter/rv.safetensors",
    controlnet=controlnet,
    torch_dtype=torch.float16,
    feature_extractor=None,
    safety_checker=None
)

pipe = pipe.to("cuda")

ip_model = IPAdapter(
    pipe,
    'models/image_encoder/',
    'models/ip-adapter_sd15.bin',
    'cuda'
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/920 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/1.45G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/1.45G [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

You have disabled the safety checker for <class 'diffusers.pipelines.controlnet.pipeline_controlnet_inpaint.StableDiffusionControlNetInpaintPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


##Do Inference and Download the Results

In [ ]:
text_prompt = "high quality detailed photograph of a woman standing"
negative_prompt = "poor quality, bad, worst, unnatural, patched, cropped, flat, unrealistic, fade, Disfigured, cartoon, blurry, nude"

In [ ]:
!rm -r /content/newres
!mkdir /content/newres

rm: cannot remove '/content/newres': No such file or directory


In [ ]:
new_size = (512, 512)

root_dir = '/content/NTED'
result_dir = '/content/newres'

In [ ]:
for root, dirnames, files in os.walk(root_dir):
  for dirname in dirnames:
    msk_img = load_image(os.path.join(root, dirname, 'mask.jpg')).resize(new_size)
    dst_img = load_image(os.path.join(root, dirname, 'distorted.jpg')).resize(new_size)
    cny_img = load_image(os.path.join(root, dirname, 'canny.png')).resize(new_size)
    seg_img = load_image(os.path.join(root, dirname, 'segment.png')).resize(new_size)
    pos_img = load_image(os.path.join(root, dirname, 'pose.png')).resize(new_size)
    rfc_img = load_image(os.path.join(root, dirname, 'flawless.jpg')).resize(new_size)

    generated_image = ip_model.generate(
        pil_image=rfc_img,
        prompt=text_prompt,
        negative_prompt=negative_prompt,
        image=dst_img,
        mask_image=msk_img,
        control_image=[cny_img, pos_img, seg_img],
        num_inference_steps=30,
        seed=23,
        controlnet_conditioning_scale = [0.5, 1.0, 0.7],
        scale=1.0,
        strength=1.0,
        num_samples=1,
        width=384,
        height=512
    )[0]

    generated_image.save(os.path.join(result_dir, f"{dirname}.jpg"))


/usr/local/lib/python3.10/dist-packages/diffusers/pipelines/controlnet/pipeline_controlnet_inpaint.py:373: FutureWarning: `_encode_prompt()` is deprecated and it will be removed in a future version. Use `encode_prompt()` instead. Also, be aware that the output format changed from a concatenated tensor to a tuple.
  deprecate("_encode_prompt()", "1.0.0", deprecation_message, standard_warn=False)


  0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
!zip -r /content/res88923.zip /content/res88923

from google.colab import files
files.download("/content/res88923.zip")

  adding: content/res88923/ (stored 0%)
  adding: content/res88923/9.jpg (deflated 3%)
  adding: content/res88923/23.jpg (deflated 4%)
  adding: content/res88923/199.jpg (deflated 4%)
  adding: content/res88923/168.jpg (deflated 2%)
  adding: content/res88923/17.jpg (deflated 3%)
  adding: content/res88923/20.jpg (deflated 1%)
  adding: content/res88923/34.jpg (deflated 2%)
  adding: content/res88923/109.jpg (deflated 2%)
  adding: content/res88923/189.jpg (deflated 3%)
  adding: content/res88923/29.jpg (deflated 3%)
  adding: content/res88923/44.jpg (deflated 2%)
  adding: content/res88923/159.jpg (deflated 6%)
  adding: content/res88923/95.jpg (deflated 3%)
  adding: content/res88923/80.jpg (deflated 3%)
  adding: content/res88923/16.jpg (deflated 4%)
  adding: content/res88923/24.jpg (deflated 2%)
  adding: content/res88923/46.jpg (deflated 3%)
  adding: content/res88923/21.jpg (deflated 5%)
  adding: content/res88923/187.jpg (deflated 2%)
  adding: content/res88923/181.jpg (deflate

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##Condition Generation
The following cells can be used in case the uploaded dataset does not provide `pose` or `canny` conditions.

In [ ]:
from controlnet_aux.processor import Processor
from diffusers.utils import load_image

# options are:
# ["canny", "depth_leres", "depth_leres++", "depth_midas", "depth_zoe", "lineart_anime",
#  "lineart_coarse", "lineart_realistic", "mediapipe_face", "mlsd", "normal_bae", "normal_midas",
#  "openpose", "openpose_face", "openpose_faceonly", "openpose_full", "openpose_hand",
#  "scribble_hed, "scribble_pidinet", "shuffle", "softedge_hed", "softedge_hedsafe",
#  "softedge_pidinet", "softedge_pidsafe", "dwpose"]

def controlnet_condition(img, processor_id):
    processor = Processor(processor_id)

    processed_image = processor(img, to_pil=True)
    return processed_image

/usr/local/lib/python3.10/dist-packages/controlnet_aux/mediapipe_face/mediapipe_face_common.py:7: UserWarning: The module 'mediapipe' is not installed. The package will have limited functionality. Please install it using the command: pip install 'mediapipe'
  warnings.warn(


In [ ]:
import os
from tqdm import tqdm

image_dir = '/content/NTED'

for root, _, files in tqdm(os.walk(image_dir)):
    for file_name in files:
      if not os.path.isfile(os.path.join(root, 'canny.png')):
        img = os.path.join(root, file_name)

        canny = controlnet_condition(
            load_image(img),
            "canny"
        )
        canny.save(os.path.join(root, 'canny.png'))

        pose = controlnet_condition(
            load_image(img),
            "openpose_full"
        )
        pose.save(os.path.join(root, 'pose.png'))

0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


body_pose_model.pth:   0%|          | 0.00/209M [00:00<?, ?B/s]

hand_pose_model.pth:   0%|          | 0.00/147M [00:00<?, ?B/s]

facenet.pth:   0%|          | 0.00/154M [00:00<?, ?B/s]

2it [01:12, 36.16s/it]


In [ ]:
import os
from tqdm import tqdm

image_dir = '/content/NTED'

for dir in [35, 39, 53, 73, 94, 95, 159, 171, 173, 178, 187, 189, 191, 199]:
    img = os.path.join(image_dir, str(dir), 'flawless.jpg')

    canny = controlnet_condition(
        load_image(img),
        "canny"
    )
    canny.save(os.path.join(image_dir, str(dir), 'canny.png'))

    pose = controlnet_condition(
        load_image(img),
        "openpose_full"
    )
    pose.save(os.path.join(image_dir, str(dir), 'pose.png'))

In [ ]:
!zip -r /content/ted.zip /content/NTED

from google.colab import files
files.download("/content/ted.zip")

  adding: content/NTED/ (stored 0%)
  adding: content/NTED/81/ (stored 0%)
  adding: content/NTED/81/mask.jpg (deflated 61%)
  adding: content/NTED/81/flawless_vis.png (deflated 13%)
  adding: content/NTED/81/distorted.jpg (deflated 5%)
  adding: content/NTED/81/canny.png (deflated 2%)
  adding: content/NTED/81/flawless.jpg (deflated 0%)
  adding: content/NTED/81/pose.png (deflated 21%)
  adding: content/NTED/159/ (stored 0%)
  adding: content/NTED/159/mask.jpg (deflated 33%)
  adding: content/NTED/159/flawless_vis.png (deflated 19%)
  adding: content/NTED/159/reference_0218-01_7_additional.jpg (deflated 0%)
  adding: content/NTED/159/distorted.jpg (deflated 6%)
  adding: content/NTED/159/reference_0218-01_2_side.jpg (deflated 0%)
  adding: content/NTED/159/reference_0218-01_3_back.jpg (deflated 0%)
  adding: content/NTED/159/canny.png (deflated 3%)
  adding: content/NTED/159/flawless.jpg (deflated 0%)
  adding: content/NTED/159/pose.png (deflated 7%)
  adding: content/NTED/39/ (stored

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>